# 1. PREPROCESSING

In [1]:
# IMPORTING LIBRARIES & PACKAGES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0. --yes
import folium

print('Libraries & packages imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
# IMPORTING GEOSPATIAL DATASET FOR NEW YORK CITY

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
nyc_neighborhoods_data = newyork_data['features'] 
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
nyc_neighborhoods = pd.DataFrame(columns=column_names)
nyc_neighborhoods.shape

(0, 4)

In [3]:
# BUILDING NEW YORK CITY DATAFRAME

for data in nyc_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_neighborhoods = nyc_neighborhoods.append({'Borough': borough,
                                         'Neighborhood': neighborhood_name,
                                         'Latitude': neighborhood_lat,
                                         'Longitude': neighborhood_lon}, ignore_index = True)
    
nyc_neighborhoods.head()    

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [4]:
nyc_neighborhoods.shape

(306, 4)

In [5]:
print('dataframe has {} boroughs and {} neighborhoods.'.
      format(len(nyc_neighborhoods['Borough'].unique()), nyc_neighborhoods.shape[0]))

dataframe has 5 boroughs and 306 neighborhoods.


## 1.1. MAPPING NEW YORK CITY AND ITS BOROUGHS TO HAVE A GENERAL IDEA OF LOCATION

In [7]:
address = 'Newyork City, NY'

geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator = geolocator.geocode(address)
latitudeNYC = location.latitude
longitudeNYC = location.longitude
print('NYC coordinates: {}, {}.'.format(latitudeNYC, longitudeNYC))

NYC coordinates: 40.74725645, -73.8251968475337.


In [8]:
map_nyc = folium.Map(location=[latitudeNYC, longitudeNYC], zoom_start=10)

for lat, lng, borough, neighborhood in zip(nyc_neighborhoods['Latitude'], nyc_neighborhoods['Longitude'], 
                                           nyc_neighborhoods['Borough'], nyc_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)

map_nyc

### 1.1.1. QUEENS

In [10]:
queens_data = nyc_neighborhoods[nyc_neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [14]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude_queens = location.latitude
longitude_queens = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude_queens, longitude_queens))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [19]:
map_queens = folium.Map(location=[latitude_queens, longitude_queens], zoom_start=10)

for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

### 1.1.2. BROOKLYN

In [11]:
brooklyn_data = nyc_neighborhoods[nyc_neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [17]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude_brooklyn = location.latitude
longitude_brooklyn = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude_brooklyn, longitude_brooklyn))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [22]:
map_brooklyn = folium.Map(location=[latitude_brooklyn, longitude_brooklyn], zoom_start=10)

for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

### 1.1.3. MANHATTAN

In [12]:
manhattan_data = nyc_neighborhoods[nyc_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude_manhattan = location.latitude
longitude_manhattan = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_manhattan, longitude_manhattan))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [20]:
map_manhattan = folium.Map(location=[latitude_manhattan, longitude_manhattan], zoom_start=10)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### 1.1.4. BRONX

In [13]:
bronx_data = nyc_neighborhoods[nyc_neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [16]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude_bronx = location.latitude
longitude_bronx = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude_bronx, longitude_bronx))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


In [21]:
map_bronx = folium.Map(location=[latitude_bronx, longitude_bronx], zoom_start=10)

for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

### 1.1.5. STATEN ISLAND

In [26]:
staten_island_data = nyc_neighborhoods[nyc_neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
staten_island_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


In [25]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude_staten_island = location.latitude
longitude_staten_island = location.longitude
print('The geograpical coordinate of Staten Island are {}, {}.'.format(latitude_staten_island, longitude_staten_island))

The geograpical coordinate of Staten Island are 40.5834557, -74.1496048.


In [27]:
map_staten_island = folium.Map(location=[latitude_staten_island, longitude_staten_island], zoom_start=10)

for lat, lng, label in zip(staten_island_data['Latitude'], staten_island_data['Longitude'], staten_island_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_staten_island)  
    
map_staten_island

# 2. FILTERING BOROUGHS TOP NEIGHBORHOODS BY VENUE

### We will explore all the 5 boroughs by venues in their neighborhoods. To be consider in the second stage, the borough must have at least 5 different neighborhoods with 50 or more venues (measure for minimun size).

In [28]:
CLIENT_ID = '2O4y5VULQVQK5YLNGFFKHHNGLDLFCBDA5HFABCHRD5H4TOOO' # your Foursquare ID
CLIENT_SECRET = '5RTN3GJ5C33EHOWNBDBSZQZV4TQKE2H2LPKDCZW0ZKCBL04K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2O4y5VULQVQK5YLNGFFKHHNGLDLFCBDA5HFABCHRD5H4TOOO
CLIENT_SECRET: 5RTN3GJ5C33EHOWNBDBSZQZV4TQKE2H2LPKDCZW0ZKCBL04K


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                          
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)            
    
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

## 2.1. QUEENS

In [30]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                latitudes=queens_data['Latitude'],
                                longitudes=queens_data['Longitude'])

Found 2067 venues in 81 neighborhoods.


In [31]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [32]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,18,18,18,18,18,18
Astoria,100,100,100,100,100,100
Astoria Heights,12,12,12,12,12,12
Auburndale,19,19,19,19,19,19
Bay Terrace,38,38,38,38,38,38
Bayside,72,72,72,72,72,72
Bayswater,3,3,3,3,3,3
Beechhurst,12,12,12,12,12,12
Bellaire,12,12,12,12,12,12


In [33]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))

There are 1698 distinct venues in 268 categories.


In [34]:
queens_venues_sorted = queens_venues.groupby('Neighborhood').count()
queens_venues_sorted = queens_venues_sorted.sort_values('Venue', ascending=False)
queens_venues_sorted_top = queens_venues_sorted[queens_venues_sorted.Venue > 50]
queens_venues_sorted_top

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Sunnyside Gardens,100,100,100,100,100,100
Astoria,100,100,100,100,100,100
Jackson Heights,80,80,80,80,80,80
Bayside,72,72,72,72,72,72
Woodside,70,70,70,70,70,70
Long Island City,68,68,68,68,68,68
Flushing,61,61,61,61,61,61
Hunters Point,52,52,52,52,52,52
Rockaway Beach,51,51,51,51,51,51


In [35]:
qt = queens_venues_sorted_top.index.to_list()
qt

['Sunnyside Gardens',
 'Astoria',
 'Jackson Heights',
 'Bayside',
 'Woodside',
 'Long Island City',
 'Flushing',
 'Hunters Point',
 'Rockaway Beach',
 'Little Neck']

In [36]:
queens_top_neighborhoods = queens_data[queens_data['Neighborhood'].isin([
 'Sunnyside Gardens',
 'Astoria',
 'Jackson Heights',
 'Woodside',
 'Bayside',
 'Long Island City',
 'Flushing',
 'Hunters Point',
 'Rockaway Beach'])]

queens_top_neighborhoods.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Flushing,40.764454,-73.831773
4,Queens,Long Island City,40.750217,-73.939202
5,Queens,Bayside,40.766041,-73.774274
6,Queens,Rockaway Beach,40.582802,-73.822361
7,Queens,Hunters Point,40.743414,-73.953868
8,Queens,Sunnyside Gardens,40.745652,-73.918193


### Obs: Queens has 9 top neighborhoods. 

## 2.2. MANHATTAN

In [37]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'])

Found 2975 venues in 40 neighborhoods.


In [38]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [39]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,56,56,56,56,56,56
Carnegie Hill,89,89,89,89,89,89
Central Harlem,42,42,42,42,42,42
Chelsea,99,99,99,99,99,99
Chinatown,100,100,100,100,100,100
Civic Center,88,88,88,88,88,88
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


In [40]:
print('There are {} distinct venues in {} categories.'.format(
    len(manhattan_venues['Venue'].unique()),len(manhattan_venues['Venue Category'].unique())))

There are 2502 distinct venues in 322 categories.


In [41]:
manhattan_venues_sorted = manhattan_venues.groupby('Neighborhood').count()
manhattan_venues_sorted = manhattan_venues_sorted.sort_values('Venue', ascending=False)
manhattan_venues_sorted_top = manhattan_venues_sorted[manhattan_venues_sorted.Venue > 50]
manhattan_venues_sorted_top

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Yorkville,100,100,100,100,100,100
Lenox Hill,100,100,100,100,100,100
West Village,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
Midtown,100,100,100,100,100,100
East Village,100,100,100,100,100,100
Financial District,100,100,100,100,100,100
Turtle Bay,100,100,100,100,100,100


In [42]:
mt = manhattan_venues_sorted_top.index.to_list()
mt

['Yorkville',
 'Lenox Hill',
 'West Village',
 'Chinatown',
 'Clinton',
 'Midtown',
 'East Village',
 'Financial District',
 'Turtle Bay',
 'Little Italy',
 'Greenwich Village',
 'Chelsea',
 'Noho',
 'Lincoln Square',
 'Sutton Place',
 'Flatiron',
 'Washington Heights',
 'Carnegie Hill',
 'Civic Center',
 'Midtown South',
 'Upper East Side',
 'Tudor City',
 'Murray Hill',
 'Soho',
 'Tribeca',
 'Upper West Side',
 'Gramercy',
 'Hamilton Heights',
 'Inwood',
 'Battery Park City']

In [43]:
manhattan_top_neighborhoods = manhattan_data[manhattan_data['Neighborhood'].isin([
 'Yorkville',
 'Greenwich Village',
 'West Village',
 'Chelsea',
 'Chinatown',
 'Midtown',
 'Clinton',
 'Turtle Bay',
 'East Village',
 'Financial District',
 'Little Italy',
 'Lenox Hill',
 'Noho',
 'Flatiron',
 'Sutton Place',
 'Lincoln Square',
 'Washington Heights',
 'Upper East Side',
 'Carnegie Hill',
 'Civic Center',
 'Midtown South',
 'Murray Hill',
 'Tudor City',
 'Soho',
 'Tribeca',
 'Upper West Side',
 'Gramercy',
 'Hamilton Heights',
 'Battery Park City',
 'Inwood',
 'Hudson Yards'])]

manhattan_top_neighborhoods.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Chinatown,40.715618,-73.994279
1,Manhattan,Washington Heights,40.851903,-73.936900
2,Manhattan,Inwood,40.867684,-73.921210
3,Manhattan,Hamilton Heights,40.823604,-73.949688
4,Manhattan,Upper East Side,40.775639,-73.960508
5,Manhattan,Yorkville,40.775930,-73.947118
6,Manhattan,Lenox Hill,40.768113,-73.958860
7,Manhattan,Upper West Side,40.787658,-73.977059
8,Manhattan,Lincoln Square,40.773529,-73.985338
9,Manhattan,Clinton,40.759101,-73.996119


### Obs: Manhattan has 31 top neighborhoods.

## 2.3. BROOKLYN

In [44]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude'])

Found 2693 venues in 70 neighborhoods.


In [45]:
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore


In [46]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,50,50,50,50,50,50
Bay Ridge,83,83,83,83,83,83
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,33,33,33,33,33,33
Bergen Beach,6,6,6,6,6,6
Boerum Hill,96,96,96,96,96,96
Borough Park,19,19,19,19,19,19
Brighton Beach,45,45,45,45,45,45
Broadway Junction,17,17,17,17,17,17


In [47]:
print('There are {} distinct venues in {} categories.'.format(
    len(brooklyn_venues['Venue'].unique()),len(brooklyn_venues['Venue Category'].unique())))

There are 2188 distinct venues in 282 categories.


In [48]:
brooklyn_venues_sorted = brooklyn_venues.groupby('Neighborhood').count()
brooklyn_venues_sorted = brooklyn_venues_sorted.sort_values('Venue', ascending=False)
brooklyn_venues_sorted_top = brooklyn_venues_sorted[brooklyn_venues_sorted.Venue > 50]
brooklyn_venues_sorted_top

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brooklyn Heights,100,100,100,100,100,100
Carroll Gardens,100,100,100,100,100,100
South Side,100,100,100,100,100,100
Cobble Hill,100,100,100,100,100,100
North Side,100,100,100,100,100,100
Boerum Hill,96,96,96,96,96,96
Clinton Hill,92,92,92,92,92,92
Downtown,89,89,89,89,89,89
Greenpoint,88,88,88,88,88,88


In [49]:
bn = brooklyn_venues_sorted_top.index.to_list()
bn

['Brooklyn Heights',
 'Carroll Gardens',
 'South Side',
 'Cobble Hill',
 'North Side',
 'Boerum Hill',
 'Clinton Hill',
 'Downtown',
 'Greenpoint',
 'Bay Ridge',
 'Bushwick',
 'Fort Hamilton',
 'East Williamsburg',
 'Gowanus',
 'Prospect Heights',
 'Fort Greene',
 'Park Slope',
 'Fulton Ferry',
 'Dumbo']

In [50]:
brooklyn_top_neighborhoods = brooklyn_data[brooklyn_data['Neighborhood'].isin([
 'South Side',
 'Carroll Gardens',
 'North Side',
 'Brooklyn Heights',
 'Cobble Hill',
 'Boerum Hill',
 'Clinton Hill',
 'Downtown',
 'Greenpoint',
 'Bay Ridge',
 'East Williamsburg',
 'Bushwick',
 'Fort Hamilton',
 'Gowanus',
 'Prospect Heights',
 'Fort Greene',
 'Park Slope',
 'Ditmas Park',
 'Fulton Ferry',
 'Dumbo'])]

brooklyn_top_neighborhoods.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Greenpoint,40.730201,-73.954241
2,Brooklyn,Prospect Heights,40.676822,-73.964859
3,Brooklyn,Bushwick,40.698116,-73.925258
4,Brooklyn,Brooklyn Heights,40.695864,-73.993782
5,Brooklyn,Cobble Hill,40.687920,-73.998561
6,Brooklyn,Carroll Gardens,40.680540,-73.994654
7,Brooklyn,Gowanus,40.673931,-73.994441
8,Brooklyn,Fort Greene,40.688527,-73.972906
9,Brooklyn,Park Slope,40.672321,-73.977050


### Obs: Brooklyn has 20 top neighborhoods.

## 2.4. BRONX

In [51]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude'])

Found 1200 venues in 52 neighborhoods.


In [52]:
bronx_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [53]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,31,31,31,31,31,31
Baychester,20,20,20,20,20,20
Bedford Park,37,37,37,37,37,37
Belmont,98,98,98,98,98,98
Bronxdale,14,14,14,14,14,14
Castle Hill,7,7,7,7,7,7
City Island,28,28,28,28,28,28
Claremont Village,18,18,18,18,18,18
Clason Point,9,9,9,9,9,9


In [54]:
print('There are {} distinct venues in {} categories.'.format(
    len(bronx_venues['Venue'].unique()),len(bronx_venues['Venue Category'].unique())))

There are 864 distinct venues in 163 categories.


In [55]:
bronx_venues_sorted = bronx_venues.groupby('Neighborhood').count()
bronx_venues_sorted = bronx_venues_sorted.sort_values('Venue', ascending=False)
bronx_venues_sorted_top = bronx_venues_sorted[bronx_venues_sorted.Venue > 50]
bronx_venues_sorted_top

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belmont,98,98,98,98,98,98
Fordham,82,82,82,82,82,82
Kingsbridge,68,68,68,68,68,68


### Obs: Bronx has only 3 top neighborhoods.

## 2.5. STATEN ISLAND

In [56]:
staten_island_venues = getNearbyVenues(names=staten_island_data['Neighborhood'],
                                   latitudes=staten_island_data['Latitude'],
                                   longitudes=staten_island_data['Longitude'])

Found 819 venues in 63 neighborhoods.


In [57]:
staten_island_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
3,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
4,St. George,40.644982,-74.079353,Shake Shack,40.643660,-74.075891,Burger Joint


In [58]:
staten_island_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,9,9,9,9,9,9
Arden Heights,6,6,6,6,6,6
Arlington,7,7,7,7,7,7
Arrochar,21,21,21,21,21,21
Bay Terrace,10,10,10,10,10,10
Bloomfield,5,5,5,5,5,5
Bulls Head,45,45,45,45,45,45
Butler Manor,6,6,6,6,6,6
Castleton Corners,13,13,13,13,13,13


In [59]:
print('There are {} distinct venues in {} categories.'.format(
    len(staten_island_venues['Venue'].unique()),len(staten_island_venues['Venue Category'].unique())))

There are 676 distinct venues in 176 categories.


In [60]:
staten_island_venues_sorted = staten_island_venues.groupby('Neighborhood').count()
staten_island_venues_sorted = staten_island_venues_sorted.sort_values('Venue', ascending=False)
staten_island_venues_sorted_top = staten_island_venues_sorted[staten_island_venues_sorted.Venue > 50]
staten_island_venues_sorted_top

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,


### Obs: Staten Island has none top neighborhood.

### CONCLUSIONS:
###                              -IN TOTAL 60 NEIGHBORHOODS OF QUEENS, MANHATTAN AND BROOKLYN MADE THE CUT.
###                              -BRONX AND STATEN DO NOT MEET CRITERIA, SO THEY ARE OUT OF THE PROCESS.

# 3. CLUSTERING NEW YORK CITY TOP NEIGHBORHOODS.

In [63]:
# BUILDING A NEW DATAFRAME WITH THE 60 TOP NEIGHBORHOODS

nyc_top_neighborhoods = pd.concat([queens_top_neighborhoods, manhattan_top_neighborhoods, brooklyn_top_neighborhoods], axis=0)
nyc_top_neighborhoods.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Flushing,40.764454,-73.831773
4,Queens,Long Island City,40.750217,-73.939202
5,Queens,Bayside,40.766041,-73.774274
6,Queens,Rockaway Beach,40.582802,-73.822361
7,Queens,Hunters Point,40.743414,-73.953868
8,Queens,Sunnyside Gardens,40.745652,-73.918193
9,Manhattan,Chinatown,40.715618,-73.994279


## 3.1. EXPLORING VENUES

In [64]:
nyc_venues = getNearbyVenues(names = nyc_top_neighborhoods['Neighborhood'],
                             latitudes = nyc_top_neighborhoods['Latitude'],
                             longitudes = nyc_top_neighborhoods['Longitude'])

Found 4860 venues in 60 neighborhoods.


In [65]:
nyc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [66]:
nyc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Astoria,100,100,100,100,100,100
Battery Park City,56,56,56,56,56,56
Bay Ridge,83,83,83,83,83,83
Bayside,72,72,72,72,72,72
Boerum Hill,96,96,96,96,96,96
Brooklyn Heights,100,100,100,100,100,100
Bushwick,73,73,73,73,73,73
Carnegie Hill,89,89,89,89,89,89
Carroll Gardens,100,100,100,100,100,100


In [67]:
print('There are {} distinct venues in {} categories.'.format(
    len(nyc_venues['Venue'].unique()),len(nyc_venues['Venue Category'].unique())))

There are 4037 distinct venues in 358 categories.


In [68]:
# ONE HOT ENCODING

nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

neighbor = nyc_onehot['Neighborhood']
nyc_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
nyc_onehot.insert(0, 'Neighborhood', neighbor)

nyc_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Renta

In [69]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Renta

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))  

In [70]:
nyc_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
nyc_neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    nyc_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

nyc_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Middle Eastern Restaurant,Bar,Hookah Bar,Greek Restaurant,Mediterranean Restaurant,Indian Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,Food Truck
1,Battery Park City,Park,Hotel,Gym,Memorial Site,Boat or Ferry,Wine Shop,Gourmet Shop,Plaza,Playground,Coffee Shop
2,Bay Ridge,Spa,Pizza Place,Italian Restaurant,Greek Restaurant,American Restaurant,Bar,Thai Restaurant,Bagel Shop,Grocery Store,Sandwich Place
3,Bayside,Bar,Indian Restaurant,Pizza Place,Sushi Restaurant,American Restaurant,Pub,Spa,Donut Shop,Café,Mexican Restaurant
4,Boerum Hill,Coffee Shop,Bar,Dance Studio,French Restaurant,Spa,Arts & Crafts Store,Furniture / Home Store,Bakery,Sandwich Place,Middle Eastern Restaurant
5,Brooklyn Heights,Yoga Studio,Deli / Bodega,Park,Bakery,Gym,Cosmetics Shop,Mexican Restaurant,Pizza Place,Italian Restaurant,Diner
6,Bushwick,Bar,Mexican Restaurant,Deli / Bodega,Coffee Shop,Pizza Place,Bakery,Thrift / Vintage Store,Discount Store,Chinese Restaurant,Vegetarian / Vegan Restaurant
7,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Bar,Italian Restaurant,Gym / Fitness Center,Japanese Restaurant,Gym,Grocery Store,Pizza Place
8,Carroll Gardens,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Wine Shop,Spa,Bar,Gourmet Shop,Café
9,Chelsea,Art Gallery,Coffee Shop,Seafood Restaurant,Ice Cream Shop,Italian Restaurant,Park,Juice Bar,Bookstore,Cycle Studio,Cupcake Shop


## 3.2. KMEANS ON TOP NEIGHBORHOODS

In [73]:
kclusters = 5

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(nyc_grouped_clustering)

kmeans.labels_

array([1, 3, 1, 1, 2, 1, 2, 2, 3, 3, 1, 3, 3, 1, 2, 1, 1, 3, 2, 2, 3, 3,
       1, 3, 1, 3, 2, 3, 2, 3, 2, 3, 3, 1, 1, 3, 3, 1, 2, 3, 0, 1, 3, 2,
       3, 2, 4, 3, 2, 1, 3, 3, 1, 3, 3, 3, 1, 3, 1, 3], dtype=int32)

In [76]:
nyc_merged = nyc_top_neighborhoods

nyc_merged['Cluster Labels'] = kmeans.labels_

nyc_merged = nyc_merged.join(nyc_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyc_merged.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,1,Middle Eastern Restaurant,Bar,Hookah Bar,Greek Restaurant,Mediterranean Restaurant,Indian Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,Food Truck
1,Queens,Woodside,40.746349,-73.901842,3,Grocery Store,Bakery,Thai Restaurant,Latin American Restaurant,Filipino Restaurant,American Restaurant,Bar,Donut Shop,Pub,Pizza Place
2,Queens,Jackson Heights,40.751981,-73.882821,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Thai Restaurant,Mobile Phone Shop,Grocery Store,Mexican Restaurant,Kids Store,Supplement Shop
3,Queens,Flushing,40.764454,-73.831773,1,Chinese Restaurant,Hotpot Restaurant,Bubble Tea Shop,Karaoke Bar,Korean Restaurant,Bakery,Gym,Construction & Landscaping,Café,Gym / Fitness Center
4,Queens,Long Island City,40.750217,-73.939202,2,Hotel,Coffee Shop,Pizza Place,Bar,Mexican Restaurant,Café,Deli / Bodega,Burger Joint,Market,Supermarket
5,Queens,Bayside,40.766041,-73.774274,1,Bar,Indian Restaurant,Pizza Place,Sushi Restaurant,American Restaurant,Pub,Spa,Donut Shop,Café,Mexican Restaurant
6,Queens,Rockaway Beach,40.582802,-73.822361,2,Beach,Ice Cream Shop,BBQ Joint,Pizza Place,Seafood Restaurant,Food Truck,Metro Station,Latin American Restaurant,Deli / Bodega,Arepa Restaurant
7,Queens,Hunters Point,40.743414,-73.953868,2,Café,Japanese Restaurant,Italian Restaurant,Wine Shop,Scenic Lookout,American Restaurant,Coffee Shop,Brewery,Gym / Fitness Center,Massage Studio
8,Queens,Sunnyside Gardens,40.745652,-73.918193,3,Bar,Grocery Store,Pizza Place,Thai Restaurant,Coffee Shop,Pharmacy,American Restaurant,Turkish Restaurant,Mexican Restaurant,Korean Restaurant
9,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Coffee Shop,Optical Shop,American Restaurant,Spa,Dumpling Restaurant,Greek Restaurant


## 3.3. MAPPING AND EXPLORING CLUSTERS

In [78]:
map_clusters = folium.Map(location=[latitudeNYC, longitudeNYC], zoom_start=10)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'],nyc_merged['Longitude'],nyc_merged['Neighborhood'],nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [79]:
nyc_cluster_0 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(4, nyc_merged.shape[1]))]]
nyc_cluster_1 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 1, nyc_merged.columns[[1] + list(range(4, nyc_merged.shape[1]))]]
nyc_cluster_2 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 2, nyc_merged.columns[[1] + list(range(4, nyc_merged.shape[1]))]]
nyc_cluster_3 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 3, nyc_merged.columns[[1] + list(range(4, nyc_merged.shape[1]))]]
nyc_cluster_4 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 4, nyc_merged.columns[[1] + list(range(4, nyc_merged.shape[1]))]]

In [80]:
nyc_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,0,Spa,Pizza Place,Italian Restaurant,Greek Restaurant,American Restaurant,Bar,Thai Restaurant,Bagel Shop,Grocery Store,Sandwich Place


In [82]:
nyc_cluster_1.reset_index(drop=True)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,1,Middle Eastern Restaurant,Bar,Hookah Bar,Greek Restaurant,Mediterranean Restaurant,Indian Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,Food Truck
1,Jackson Heights,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Thai Restaurant,Mobile Phone Shop,Grocery Store,Mexican Restaurant,Kids Store,Supplement Shop
2,Flushing,1,Chinese Restaurant,Hotpot Restaurant,Bubble Tea Shop,Karaoke Bar,Korean Restaurant,Bakery,Gym,Construction & Landscaping,Café,Gym / Fitness Center
3,Bayside,1,Bar,Indian Restaurant,Pizza Place,Sushi Restaurant,American Restaurant,Pub,Spa,Donut Shop,Café,Mexican Restaurant
4,Washington Heights,1,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Pizza Place,Supplement Shop,Coffee Shop,Bank,Spanish Restaurant
5,Upper East Side,1,Italian Restaurant,Juice Bar,Gym / Fitness Center,Exhibit,Bakery,Yoga Studio,Wine Shop,Hotel,American Restaurant,Cosmetics Shop
6,Lenox Hill,1,Italian Restaurant,Pizza Place,Coffee Shop,Sushi Restaurant,Cocktail Bar,Café,Gym / Fitness Center,Gym,Burger Joint,Bakery
7,Upper West Side,1,Italian Restaurant,Bakery,Wine Bar,Coffee Shop,Dessert Shop,Bagel Shop,Bar,Mediterranean Restaurant,Pizza Place,Ice Cream Shop
8,Greenwich Village,1,Italian Restaurant,Coffee Shop,Café,Bakery,Wine Bar,Gym,Sushi Restaurant,Burger Joint,Caribbean Restaurant,Comedy Club
9,Tribeca,1,Park,Italian Restaurant,Café,Wine Bar,Spa,Bakery,Scenic Lookout,Skate Park,Men's Store,Art Gallery


In [83]:
nyc_cluster_2.reset_index(drop=True)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Long Island City,2,Hotel,Coffee Shop,Pizza Place,Bar,Mexican Restaurant,Café,Deli / Bodega,Burger Joint,Market,Supermarket
1,Rockaway Beach,2,Beach,Ice Cream Shop,BBQ Joint,Pizza Place,Seafood Restaurant,Food Truck,Metro Station,Latin American Restaurant,Deli / Bodega,Arepa Restaurant
2,Hunters Point,2,Café,Japanese Restaurant,Italian Restaurant,Wine Shop,Scenic Lookout,American Restaurant,Coffee Shop,Brewery,Gym / Fitness Center,Massage Studio
3,Yorkville,2,Coffee Shop,Italian Restaurant,Gym,Bar,Deli / Bodega,Sushi Restaurant,Diner,Mexican Restaurant,Japanese Restaurant,Wine Shop
4,Clinton,2,Theater,Gym / Fitness Center,Coffee Shop,Wine Shop,Gym,Hotel,Italian Restaurant,Sandwich Place,American Restaurant,Spa
5,Midtown,2,Hotel,Clothing Store,Coffee Shop,Theater,Cuban Restaurant,Pizza Place,Spa,Salon / Barbershop,French Restaurant,Bookstore
6,Soho,2,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,French Restaurant,Spa,Gym,Clothing Store,Café,Sandwich Place,Pilates Studio
7,Gramercy,2,Coffee Shop,Italian Restaurant,Grocery Store,Pizza Place,Playground,Bar,Spa,Bagel Shop,Sandwich Place,Taco Place
8,Financial District,2,Coffee Shop,Hotel,Gym / Fitness Center,Salad Place,Cocktail Bar,Park,American Restaurant,Pizza Place,Falafel Restaurant,Japanese Restaurant
9,Flatiron,2,Gym / Fitness Center,Yoga Studio,Outdoor Sculpture,Cosmetics Shop,Park,Wine Shop,Italian Restaurant,Cheese Shop,Mediterranean Restaurant,Spa


In [84]:
nyc_cluster_3.reset_index(drop=True)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Woodside,3,Grocery Store,Bakery,Thai Restaurant,Latin American Restaurant,Filipino Restaurant,American Restaurant,Bar,Donut Shop,Pub,Pizza Place
1,Sunnyside Gardens,3,Bar,Grocery Store,Pizza Place,Thai Restaurant,Coffee Shop,Pharmacy,American Restaurant,Turkish Restaurant,Mexican Restaurant,Korean Restaurant
2,Chinatown,3,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Coffee Shop,Optical Shop,American Restaurant,Spa,Dumpling Restaurant,Greek Restaurant
3,Inwood,3,Mexican Restaurant,Café,Bakery,Lounge,Pizza Place,Restaurant,Chinese Restaurant,Wine Bar,Pharmacy,American Restaurant
4,Hamilton Heights,3,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Yoga Studio,Chinese Restaurant,Bakery,Sushi Restaurant,Caribbean Restaurant
5,Lincoln Square,3,Gym / Fitness Center,Plaza,Café,Theater,Italian Restaurant,Concert Hall,Performing Arts Venue,American Restaurant,Wine Shop,Indie Movie Theater
6,Murray Hill,3,Sandwich Place,Japanese Restaurant,Coffee Shop,Burger Joint,Steakhouse,Pizza Place,Chinese Restaurant,Gym / Fitness Center,Bar,Bakery
7,Chelsea,3,Art Gallery,Coffee Shop,Seafood Restaurant,Ice Cream Shop,Italian Restaurant,Park,Juice Bar,Bookstore,Cycle Studio,Cupcake Shop
8,East Village,3,Pizza Place,Coffee Shop,Juice Bar,Japanese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Cocktail Bar,Seafood Restaurant,Italian Restaurant,Bar
9,Little Italy,3,Chinese Restaurant,Italian Restaurant,Spa,Pizza Place,Bubble Tea Shop,Bakery,Mediterranean Restaurant,Hotel,Café,Thai Restaurant


In [87]:
nyc_cluster_4.reset_index(drop=True)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Carroll Gardens,4,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Wine Shop,Spa,Bar,Gourmet Shop,Café


# 4. CONCLUSION

### We can see that Cluster 2 is definitively more appealing than the other ones for Headquartering the company.  It has a good variety of venues for servicing and entertaining the employees and a good presence of hotels availables for temporary visitors, corporate meetings, etc. Neighborhoods on Cluster 2 look more commercial than the other ones, that have a more residential character, which is what we are looking for. So the following 12 neighborhoods should be consider on the next stage of the evaluation for locating headquarters: Long Island City, Rockaway Beach, Hunters Point, Yorkvile, Clinton, Midtown, Soho, Gramercy, Financial District, Flatiron, Buschwick, Cobblehill and Fort Greene.